## Introduction

This is the first in a series of lessons related to astronomy data.

As a running example, we will replicate part of the analysis in a recent paper, "[Off the beaten path: Gaia reveals GD-1 stars outside of the main stream](https://arxiv.org/abs/1805.00425)" by Adrian M. Price-Whelan and Ana Bonaca.

As the abstract explains, "Using data from the Gaia second data release combined with Pan-STARRS photometry, we present a sample of highly-probable members of the longest cold stream in the Milky Way, GD-1."

GD-1 is a [stellar stream](https://en.wikipedia.org/wiki/List_of_stellar_streams) which is "an association of stars orbiting a galaxy that was once a globular cluster or dwarf galaxy that has now been torn apart and stretched out along its orbit by tidal forces."

[This article](https://www.sciencemag.org/news/2018/10/streams-stars-reveal-galaxy-s-violent-history-and-perhaps-its-unseen-dark-matter) explains some of the background of this paper, including the process that led to the paper and an discussion of the scientific implications:

* "The streams are particularly useful for ... galactic archaeology—rewinding the cosmic clock to reconstruct the assembly of the Milky Way."

* "They also are being used as exquisitely sensitive scales to measure the galaxy's mass."

* "... the streams are well-positioned to reveal the presence of dark matter ... because the streams are so fragile, theorists say, collisions with marauding clumps of dark matter could leave telltale scars, potential clues to its nature."

In this lesson, we will replicate the first step in their analysis, downloading Gaia data using a database query language called ADQL.

## Using ADQL to download Gaia data

The two datasets used in this study are
 
* [Gaia](https://en.wikipedia.org/wiki/Gaia_(spacecraft)), which is "a space observatory of the European Space Agency (ESA), launched in 2013 ... designed for astrometry: measuring the positions, distances and motions of stars with unprecedented precision", and

* [Pan-STARRS](https://en.wikipedia.org/wiki/Pan-STARRS): The Panoramic Survey Telescope and Rapid Response System is a survey designed to monitor the sky for transient objects, producing a catalog with accurate astronometry and photometry of detected sources."

Both of these datasets are very large, which can make them challenging to work with.  It might not be possible, or practical, to download the entire dataset.

One of the goals of this workshop is to provide tools for working with large datasets.  One of the most important of those tools is a "query language", which is a way to query a large database and efficiently select the information you need.  So that's where we'll start.

The query language we'll use is ADQL, which stands for "Astronomical Data Query Language".

ADQL is a dialect of [SQL](https://en.wikipedia.org/wiki/SQL) (Structured Query Language), which is by far the most commonly used query language.  Almost everything you learn about ADQL also works in SQL.

[The reference manual for ADQL is here](http://www.ivoa.net/documents/ADQL/20180112/PR-ADQL-2.1-20180112.html).

But you might find it easier to learn from [this ADQL Cookbook](https://www.gaia.ac.uk/data/gaia-data-release-1/adql-cookbook).

## Installing libraries

The library we'll use to get Gaia data is [Astroquery](https://astroquery.readthedocs.io/en/latest/).

If you are running this notebook on Colab, you can run the following cell to install Astroquery and a the other libraries we'll use.

If you are running this notebook on your own computer, you might have to install these libraries yourself.  

If you are using this notebook as part of a Carpentries workshop, you should have received setup instructions.

TODO: Add a link to the instructions.


In [2]:
# If we're running on Colab, install libraries

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install astroquery astro-gala pyia

## Outline

The next few sections demonstrate the steps for selecting and downloading data from the Gaia Database:

1. First we'll make a connection to the Gaia server,

2. We will explore information about the databased and the tables it contains,

3. We will write a query and send it to the server, and finally

4. We will download the response from the server.

## Connecting to Gaia

Astroquery provides `Gaia`, which is an [object that represents a connection to the Gaia database](https://astroquery.readthedocs.io/en/latest/gaia/gaia.html).

We can connect to the Gaia database like this:

In [3]:
from astroquery.gaia import Gaia

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


#### Optional detail 

> Running this import statement has the effect of creating a [TAP+](http://www.ivoa.net/documents/TAP/) connection; TAP stands for "Table Access Protocol".  It is a network protocol for sending queries to the database and getting back the results.  I'm not sure why it seems to create two connections.

## Databases and Tables

What is a database, anyway?  Most generally, it can be any collection of data, but when we are talking about ADQL or SQL:

* A database is a collection of one or more named tables.

* Each table is a 2-D array with one or more named columns of data.

We can use `load_tables` to get the names of the tables in the Gaia database.  With the option `only_names=True`, it loads information about the tables, called the "metadata", not the data itself.

In [4]:
tables = Gaia.load_tables(only_names=True)
for table in (tables):
    print(table.get_qualified_name())

INFO: Retrieving tables... [astroquery.utils.tap.core]
INFO: Parsing tables... [astroquery.utils.tap.core]
INFO: Done. [astroquery.utils.tap.core]
external.external.apassdr9
external.external.gaiadr2_geometric_distance
external.external.galex_ais
external.external.ravedr5_com
external.external.ravedr5_dr5
external.external.ravedr5_gra
external.external.ravedr5_on
external.external.sdssdr13_photoprimary
external.external.skymapperdr1_master
external.external.tmass_xsc
public.public.hipparcos
public.public.hipparcos_newreduction
public.public.hubble_sc
public.public.igsl_source
public.public.igsl_source_catalog_ids
public.public.tycho2
public.public.dual
tap_config.tap_config.coord_sys
tap_config.tap_config.properties
tap_schema.tap_schema.columns
tap_schema.tap_schema.key_columns
tap_schema.tap_schema.keys
tap_schema.tap_schema.schemas
tap_schema.tap_schema.tables
gaiadr1.gaiadr1.aux_qso_icrf2_match
gaiadr1.gaiadr1.ext_phot_zero_point
gaiadr1.gaiadr1.allwise_best_neighbour
gaiadr1.gaiad

So that's a lot of tables.  The ones we'll use are:

* `gaiadr2.gaia_source`, which contains Gaia data from [data release 2](https://www.cosmos.esa.int/web/gaia/data-release-2),

* `gaiadr2.panstarrs1_original_valid`, which contains the photometry data we'll use from PanSTARRS, and

* `gaiadr2.panstarrs1_best_neighbour`, which we'll use to cross-match each star observed by Gaia with the same star observed by PanSTARRS.

We can use `load_table` (not `load_tables`) to get the metadata for a single table.  The name of this function is misleading, because it only downloads metadata. 

In [6]:
meta = Gaia.load_table('gaiadr2.gaia_source')
meta

Retrieving table 'gaiadr2.gaia_source'
Parsing table 'gaiadr2.gaia_source'...
Done.


Jupyter shows that the result is an object of type `TapTableMeta`, but it does not display the contents.

To see the metadata, we have to print the object.

In [5]:
print(meta)

TAP Table name: gaiadr2.gaiadr2.gaia_source
Description: This table has an entry for every Gaia observed source as listed in the
Main Database accumulating catalogue version from which the catalogue
release has been generated. It contains the basic source parameters,
that is only final data (no epoch data) and no spectra (neither final
nor epoch).
Num. columns: 96


Notice one gotcha: in the list of table names, this table appears as `gaiadr2.gaiadr2.gaia_source`, but when we load the metadata, we refer to it as `gaiadr2.gaia_source`.

**Exercise:** Go back and try

```
meta = Gaia.load_table('gaiadr2.gaiadr2.gaia_source')
```

What happens?  Is the error message helpful?  If you had not made this error deliberately, would you have been able to figure it out?

The following loop prints the names of the columns in the table.

In [6]:
for column in meta.columns:
    print(column.name)

solution_id
designation
source_id
random_index
ref_epoch
ra
ra_error
dec
dec_error
parallax
parallax_error
parallax_over_error
pmra
pmra_error
pmdec
pmdec_error
ra_dec_corr
ra_parallax_corr
ra_pmra_corr
ra_pmdec_corr
dec_parallax_corr
dec_pmra_corr
dec_pmdec_corr
parallax_pmra_corr
parallax_pmdec_corr
pmra_pmdec_corr
astrometric_n_obs_al
astrometric_n_obs_ac
astrometric_n_good_obs_al
astrometric_n_bad_obs_al
astrometric_gof_al
astrometric_chi2_al
astrometric_excess_noise
astrometric_excess_noise_sig
astrometric_params_solved
astrometric_primary_flag
astrometric_weight_al
astrometric_pseudo_colour
astrometric_pseudo_colour_error
mean_varpi_factor_al
astrometric_matched_observations
visibility_periods_used
astrometric_sigma5d_max
frame_rotator_object_type
matched_observations
duplicated_source
phot_g_n_obs
phot_g_mean_flux
phot_g_mean_flux_error
phot_g_mean_flux_over_error
phot_g_mean_mag
phot_bp_n_obs
phot_bp_mean_flux
phot_bp_mean_flux_error
phot_bp_mean_flux_over_error
phot_bp_mean_ma

You can probably guess what many of these columns are by looking at the names, but you should resist the temptation to guess.

To find out what the columns mean, [read the documentation](https://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html).

If you want to know what can go wrong when you don't read the documentation, [you might like this article](https://www.vox.com/future-perfect/2019/6/4/18650969/married-women-miserable-fake-paul-dolan-happiness).

**Exercise:** One of the other tables we'll use is `gaiadr2.gaiadr2.panstarrs1_original_valid`.  Use `load_table` to get the metadata for this table.  How many columns are there and what are their names?

Hint: Remember the gotcha we mentioned earlier.

In [7]:
# Solution

meta2 = Gaia.load_table('gaiadr2.panstarrs1_original_valid')
print(meta2)

Retrieving table 'gaiadr2.panstarrs1_original_valid'
Parsing table 'gaiadr2.panstarrs1_original_valid'...
Done.
TAP Table name: gaiadr2.gaiadr2.panstarrs1_original_valid
Description: The Panoramic Survey Telescope and Rapid Response System (Pan-STARRS) is
a system for wide-field astronomical imaging developed and operated by
the Institute for Astronomy at the University of Hawaii. Pan-STARRS1
(PS1) is the first part of Pan-STARRS to be completed and is the basis
for Data Release 1 (DR1). The PS1 survey used a 1.8 meter telescope and
its 1.4 Gigapixel camera to image the sky in five broadband filters (g,
r, i, z, y).

The current table contains a filtered subsample of the 10 723 304 629
entries listed in the original ObjectThin table.
We used only ObjectThin and MeanObject tables to extract
panstarrs1OriginalValid table, this means that objects detected only in
stack images are not included here. The main reason for us to avoid the
use of objects detected in stack images is that their a

In [8]:
# Solution

for column in meta2.columns:
    print(column.name)

obj_name
obj_id
ra
dec
ra_error
dec_error
epoch_mean
g_mean_psf_mag
g_mean_psf_mag_error
g_flags
r_mean_psf_mag
r_mean_psf_mag_error
r_flags
i_mean_psf_mag
i_mean_psf_mag_error
i_flags
z_mean_psf_mag
z_mean_psf_mag_error
z_flags
y_mean_psf_mag
y_mean_psf_mag_error
y_flags
n_detections
zone_id
obj_info_flag
quality_flag


## Writing queries

Now you might be wondering how we actually download the data.  With tables this big, you generally don't.  Instead, you use queries to select only the data you want.

A query is a string written in a query language like SQL; for the Gaia database, the query language is a dialect of SQL called ADQL.

Here's an example of an ADQL query.

In [9]:
query1 = """SELECT 
TOP 10
source_id, ref_epoch, ra, dec, parallax 
FROM gaiadr2.gaia_source"""

**Python note:** We use a [triple-quoted string](https://docs.python.org/3/tutorial/introduction.html#strings) here so we can include line breaks in the query, which makes it easier to read.

The words in uppercase are ADQL keywords:

* `SELECT` indicates that we are selecting data (as opposed to adding or modifying data).

* `TOP` indicates that we only want the first 10 rows of the table, which is useful for testing a query before asking for all of the data.

* `FROM` specifies which table we want data from.

The third line is a list of column names, indicating which columns we want.  

The column names are in lowercase to make it clear that they are not keywords.  This is a common style, but it is not required.

To run this query, we use the `Gaia` object, which represents our connection to the Gaia database, and invoke `launch_job`:

In [10]:
job1 = Gaia.launch_job(query1)
job1

The result is an object that represents the job running on a Gaia server.

If you print it, it displays metadata for the forthcoming table.

In [11]:
print(job1)

<Table length=10>
   name    dtype  unit                            description                             n_bad
--------- ------- ---- ------------------------------------------------------------------ -----
source_id   int64      Unique source identifier (unique within a particular Data Release)     0
ref_epoch float64   yr                                                    Reference epoch     0
       ra float64  deg                                                    Right ascension     0
      dec float64  deg                                                        Declination     0
 parallax float64  mas                                                           Parallax     1
Jobid: None
Phase: COMPLETED
Owner: None
Output file: sync_20200808153205.xml.gz
Results: None


`Phase: COMPLETED` indicates that the job is complete, so we can get the results:

In [12]:
results1 = job1.get_results()
type(results1)

astropy.table.table.Table

The results is an [Astropy Table](https://docs.astropy.org/en/stable/table/) is similar to a table in an SQL database except:

* SQL databases are stored on disk drives, so they are persistent; that is, they "survive" even if you turn off the computer.  An Astropy `Table` is stored in memory; it disappears when you turn off the computer (or shut down this Jupyter notebook).

* SQL databases are designed to process queries.  And Astropy `Table` can perform some query-like operations, like selecting columns and rows.  But these operations use Python syntax, not SQL.

Jupyter knows how to display the contents of a `Table`.

In [13]:
results1

source_id,ref_epoch,ra,dec,parallax
,yr,deg,deg,mas
int64,float64,float64,float64,float64
3160242094655211136,2015.5,107.69963092285408,12.42060643169392,0.38873143801935744
3160251367488889088,2015.5,107.40002452653339,12.451877874246922,0.2816508367238212
3160251749742643328,2015.5,107.60408265382821,12.37219879095663,-0.5531848203290188
3160255799895999104,2015.5,107.67284883537286,12.544868440242192,--
3160220031407186944,2015.5,107.27258794711979,12.188873073110003,0.5391782658713742
3160203710531020160,2015.5,107.3666142834879,12.008733712651349,0.171915277891479
3160217458723187584,2015.5,107.19338979023517,12.09013764451882,0.5353718184488321
3160192303097368704,2015.5,107.34057289571422,11.831711692602303,0.8743214105702177


Each column has a name, units, and a data type.

For example, the units of `ra` and `dec` are degrees, and their data type is `float64`, which is a 64-bit floating-point number, used to store measurements with a fraction part.

This information comes from the Gaia database, and has been stored in the Astropy `Table` by Astroquery.

**Exercise:** Read [the documentation of this table](https://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html) and select at least one column name that looks interesting to you.  Add a column to the query and run it again.  What are the units of the column you selected?  What is its data type?

## Asynchronous queries

`launch_job` asks the server to run the job "synchronously", which normally means it runs immediately.  But synchronous jobs are limited to 2000 rows.  For queries that return more rows, you have to run "asynchronously", which mean they might take longer to get started.

The results of an asynchronous query are stored in a file on the server, so you can start a query and come back later to get the results.

For anonymous users, files are kept for three days.

We could run `query1` again asynchronously, but let's add a new keyword, `WHERE`:

In [14]:
query2 = """SELECT TOP 100
source_id, ref_epoch, ra, dec, parallax
FROM gaiadr2.gaia_source
WHERE parallax < 1
"""

A `WHERE` clause indicates which rows we want; in this case, the query selects only rows "where" `parallax` is less than 1.  This has the effect of selecting stars with relatively low parallax, which are farther away.  We assume that Price-Whelan and Bonaca chose this threshold to exclude nearby stars that cannot be part of GD-1.

`WHERE` is one of the most common clauses in ADQL/SQL, and one of the most useful, because it allows us to select only the rows we need from the database.

We use `launch_job_async` to submit an asynchronous query.

In [15]:
job2 = Gaia.launch_job_async(query2)
print(job2)

INFO: Query finished. [astroquery.utils.tap.core]
<Table length=100>
   name    dtype  unit                            description                            
--------- ------- ---- ------------------------------------------------------------------
source_id   int64      Unique source identifier (unique within a particular Data Release)
ref_epoch float64   yr                                                    Reference epoch
       ra float64  deg                                                    Right ascension
      dec float64  deg                                                        Declination
 parallax float64  mas                                                           Parallax
Jobid: 1596915126596O
Phase: COMPLETED
Owner: None
Output file: async_20200808153207.vot
Results: None


And here are the results.

In [16]:
results2 = job2.get_results()
results2

source_id,ref_epoch,ra,dec,parallax
,yr,deg,deg,mas
int64,float64,float64,float64,float64
3160242094655211136,2015.5,107.69963092285408,12.42060643169392,0.38873143801935744
3160251367488889088,2015.5,107.40002452653339,12.451877874246922,0.2816508367238212
3160251749742643328,2015.5,107.60408265382821,12.37219879095663,-0.5531848203290188
3160220031407186944,2015.5,107.27258794711979,12.188873073110003,0.5391782658713742
3160203710531020160,2015.5,107.3666142834879,12.008733712651349,0.171915277891479
3160217458723187584,2015.5,107.19338979023517,12.09013764451882,0.5353718184488321
3160192303097368704,2015.5,107.34057289571422,11.831711692602303,0.8743214105702177
3160258896566912384,2015.5,107.63529197756,12.568794576578695,0.37894302320548223


You might notice that some values of `parallax` are negative.  As [this FAQ explains](https://www.cosmos.esa.int/web/gaia/archive-tips#negative%20parallax), "Negative parallaxes are caused by errors in the observations."  Negative parallaxes have "no physical meaning," but they can be a "useful diagnostic on the quality of the astrometric solution."

Later we will see an example where we use `parallax` and `parallax_error` to identify stars where the distance estimate is likely to be accurate or not.

**Exercise:** The clauses in a query have to be in the right order.  Go back and change the order of the clauses in `query2` and run it again.  

They query should fail, but notice that you don't get much useful debugging information.  

For this reason, developing and debugging ADQL queries can be really hard.  A few suggestions that might help:

* Whenever possible, start with a working query, either an example you find online or a query you have used in the past.

* Make small changes and test each change before you continue.

* While you are debugging, use `TOP` to limit the number of rows in the result.  That will make each attempt run faster, which reduces your testing time.  

* Launching test queries synchronously might make them start faster, too.

**Exercise:**  In a `WHERE` clause, you can use any of the comparison operators:

* `>`: greater than
* `<`: less than
* `>=`: greater than or equal
* `<=`: less than or equal
* `=`: equal
* `<>`: not equal

Most of these are the same as Python, but some are not.  Be careful to keep your Python out of your ADQL!

You can combine comparisons using the logical operators:

* AND: true if both comparisons are true
* OR: true if either or both comparisons are true

Finally, you can use `NOT` to invert the result of a comparison. 

[Read about SQL operators here](https://www.w3schools.com/sql/sql_operators.asp) and then modify the previous query to select rows where `bp_rp` is between `-0.75` and `2`.

You can [read about this variable here](https://gea.esac.esa.int/archive/documentation/GDR2/Gaia_archive/chap_datamodel/sec_dm_main_tables/ssec_dm_gaia_source.html).

In [17]:
# Solution

# This is what most people will probably do

query = """SELECT TOP 10
source_id, ref_epoch, ra, dec, parallax
FROM gaiadr2.gaia_source
WHERE parallax < 1 
  AND bp_rp > -0.75 AND bp_rp < 2
"""

In [18]:
# Solution

# But if someone notices the BETWEEN operator, 
# they might do this

query = """SELECT TOP 10
source_id, ref_epoch, ra, dec, parallax
FROM gaiadr2.gaia_source
WHERE parallax < 1 
  AND bp_rp BETWEEN -0.75 AND 2
"""

This [Hertzsprung-Russell diagram](https://sci.esa.int/web/gaia/-/60198-gaia-hertzsprung-russell-diagram) shows the BP-RP color and luminosity of stars in the Gaia catalog.

Selecting stars with `bp-rp` less than 2 excludes many class M dwarf stars, which are low temperature, low luminosity.  A star like that at GD-1's distance would be hard to detect, so if it is detected, it it more likely to be in the foreground.

## Cleaning up

Asynchronous jobs have a `jobid`.

In [19]:
job1.jobid, job2.jobid

(None, '1596915126596O')

Which you can use to remove the job from the server.

In [20]:
Gaia.remove_jobs([job2.jobid])

Removed jobs: '['1596915126596O']'.


If you don't remove it job from the server, it will be removed eventually, so don't feel too bad if you don't clean up after yourself.

## Formatting queries

So far the queries have been string "literals", meaning that the entire string is part of the program.
But writing queries yourself can be slow, repetitive, and error-prone.

It is often a good idea two write Python code that assembles a query for you.  One useful tool for that is the [string `format` method](https://www.w3schools.com/python/ref_string_format.asp).

As an example, we'll divide the previous query into two parts; a list of column names and a "base" for the query that contains everything except the column names.

Here's the list of columns we'll select.  

In [21]:
columns = 'source_id, ra, dec, pmra, pmdec, parallax, parallax_error, radial_velocity'

And here's the base; it's a string that contains at least one format specifier in curly brackets (braces).

In [22]:
query3_base = """SELECT TOP 10 
{columns}
FROM gaiadr2.gaia_source
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2
"""

This base query contains one format specifier, `{columns}`, which is a placeholder for the list of column names we will provide.

To assemble the query, we invoke `format` on the base string and provide a keyword argument that assigns a value to `columns`.

In [23]:
query3 = query3_base.format(columns=columns)

The result is a string with line breaks.  If you display it, the line breaks appear as `\n`.

In [24]:
query3

'SELECT TOP 10 \nsource_id, ra, dec, pmra, pmdec, parallax, parallax_error, radial_velocity\nFROM gaiadr2.gaia_source\nWHERE parallax < 1\n  AND bp_rp BETWEEN -0.75 AND 2\n'

But if you print it, the line breaks appear as... line breaks.

In [25]:
print(query3)

SELECT TOP 10 
source_id, ra, dec, pmra, pmdec, parallax, parallax_error, radial_velocity
FROM gaiadr2.gaia_source
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2



Notice that the format specifier has been replaced with the value of `columns`.

Let's run it and see if it works:

In [26]:
job3 = Gaia.launch_job(query3)
print(job3)

<Table length=10>
      name       dtype    unit                              description                             n_bad
--------------- ------- -------- ------------------------------------------------------------------ -----
      source_id   int64          Unique source identifier (unique within a particular Data Release)     0
             ra float64      deg                                                    Right ascension     0
            dec float64      deg                                                        Declination     0
           pmra float64 mas / yr                         Proper motion in right ascension direction     0
          pmdec float64 mas / yr                             Proper motion in declination direction     0
       parallax float64      mas                                                           Parallax     0
 parallax_error float64      mas                                         Standard error of parallax     0
radial_velocity float64   km

In [27]:
results3 = job3.get_results()
results3

source_id,ra,dec,pmra,pmdec,parallax,parallax_error,radial_velocity
,deg,deg,mas / yr,mas / yr,mas,mas,km / s
int64,float64,float64,float64,float64,float64,float64,float64
3160242094655211136,107.69963092285408,12.42060643169392,0.21809670692831037,-0.10790476537978791,0.38873143801935744,0.0879136261621634,--
3160251367488889088,107.40002452653339,12.451877874246922,-1.3002230390219955,-0.815458223486904,0.2816508367238212,0.5376061911626673,--
3160251749742643328,107.60408265382821,12.37219879095663,3.5364051135173655,-5.31019009786042,-0.5531848203290188,0.42582055001337277,--
3160220031407186944,107.27258794711979,12.188873073110003,-1.550004006280123,0.3971508244241839,0.5391782658713742,0.8321841581464361,--
3160203710531020160,107.3666142834879,12.008733712651349,-5.6435820618833095,-2.654352442128373,0.171915277891479,0.5341103882886453,--
3160217458723187584,107.19338979023517,12.09013764451882,-4.091212951580313,-1.0279665273936596,0.5353718184488321,0.2155798819783981,--
3160192303097368704,107.34057289571422,11.831711692602303,-0.18098947455284853,-0.17391778248636072,0.8743214105702177,0.46459475878336276,--
3160258896566912384,107.63529197756,12.568794576578695,0.5669753034789353,-0.21646588418430399,0.37894302320548223,0.057719850402187634,--


Good so far.

**Exercise:** This query always selects sources with `parallax` less than 1.  But suppose you want to take that upper bound as an input.

Modify `query3_base` to replace `1` with a format specifier like `{max_parallax}`.  Now, when you call `format`, add a keyword argument that assigns a value to `max_parallax`, and confirm that the format specifier gets replaced with the value you provide.

In [28]:
# Solution

query_base = """SELECT TOP 10
{columns}
FROM gaiadr2.gaia_source
WHERE parallax < {max_parallax} AND 
bp_rp BETWEEN -0.75 AND 2
"""

In [29]:
# Solution

query = query_base.format(columns=columns,
                          max_parallax=0.5)
print(query)

SELECT TOP 10
source_id, ra, dec, pmra, pmdec, parallax, parallax_error, radial_velocity
FROM gaiadr2.gaia_source
WHERE parallax < 0.5 AND 
bp_rp BETWEEN -0.75 AND 2



**Style note:**  You might notice that the variable names in this notebook are numbered, like `query1`, `query2`, etc.  

The advantage of this style is that it isolates each section of the notebook from the others, so if you go back and run the cells out of order, it's less likely that you will get unexpected interactions.

A drawback of this style is that it can be a nuisance to update the notebook if you add, remove, or reorder a section.

What do you think of this choice?  Are there alternatives you prefer?

## Selecting a region

One of the most common ways to restrict a query is to select stars in a particular region of the sky.

For example, here's a query from the [Gaia archive documentation](https://gea.esac.esa.int/archive-help/adql/examples/index.html) that selects "all the objects ... in a circular region centered at (266.41683, -29.00781) with a search radius of 5 arcmin (0.08333 deg)."

In [30]:
query = """
SELECT 
TOP 10 source_id
FROM gaiadr2.gaia_source
WHERE 1=CONTAINS(
  POINT(ra, dec),
  CIRCLE(266.41683, -29.00781, 0.08333333))
"""

This query uses three keywords that are specific to ADQL (not SQL):

* `POINT`: a location in [ICRS coordinates](https://en.wikipedia.org/wiki/International_Celestial_Reference_System), specified in degrees of right ascension and declination.

* `CIRCLE`: a circle where the first two values are the coordinates of the center and the third is the radius in degrees.

* `CONTAINS`: a function that returns `1` if a `POINT` is contained in a shape and `0` otherwise.

Here is the [documentation of `CONTAINS`](http://www.ivoa.net/documents/ADQL/20180112/PR-ADQL-2.1-20180112.html#tth_sEc4.2.12).

A query like this is called a cone search because it selects stars in a cone.

In [31]:
job = Gaia.launch_job(query)
result = job.get_results()
result

source_id
int64
4057468321929794432
4057468287575835392
4057482027171038976
4057470349160630656
4057470039924301696
4057469868125641984
4057468351995073024
4057469661959554560
4057470520960672640


**Exercise:** When you are debugging queries like this, you can use `TOP` to limit the size of the results, but then you still don't know how big the results will be.

An alternative is to use `COUNT`, which asks for the number of rows that would be selected, but it does not return them.

In the previous query, replace `TOP 10 source_id` with `COUNT(source_id)` and run the query again.  How many stars has Gaia identified in the cone we searched?

## Getting GD-1 Data

From the Price-Whelan and Bonaca paper, we will try to reproduce Figure 1, which includes this representation of stars likely to belong to GD-1:

<img src="https://github.com/datacarpentry/astronomy-python/raw/gh-pages/fig/gd1-4.png">

Along the axis of right ascension ($\phi_1$) the figure extends from -100 to 20 degrees.

Along the axis of declination ($\phi_2$) the figure extends from about -8 to 4 degrees.

Ideally, we could select all stars from this rectangle, but there are more than 10 million of them, so

* That would be difficult to work with,

* As anonymous users, we are limited to 3 million rows in a single query, and

* While we are developing and testing code, it will be faster to work with a smaller dataset.

So we'll start by selecting stars in a smaller rectangle, from -55 to -45 degrees right ascension and -8 to 4 degrees of declination.

But first we have to figure out how to represent quantities with units like degrees.

## Working with coordinates

Coordinates are physical quantities, which means that they have two parts, a value and a unit.

For example, the coordinate $30^{\circ}$ has value 30 and its units are degrees.

Until recently, most scientific computation was done with values only; units were left out of the program altogether, [often with disastrous results](https://en.wikipedia.org/wiki/Mars_Climate_Orbiter#Cause_of_failure).

Astropy provides tools for including units explicitly in computations, which makes it possible to detect errors before they cause disasters.

To use Astropy units, we import them like this:

In [32]:
import astropy.units as u

u

<module 'astropy.units' from '/home/downey/anaconda3/envs/astronomy-python/lib/python3.8/site-packages/astropy/units/__init__.py'>

`u` is an object that contains most common units and all SI units.

You can use `dir` to list them, but you should also [read the documentation](https://docs.astropy.org/en/stable/units/).

In [33]:
dir(u)

['A',
 'AA',
 'AB',
 'ABflux',
 'ABmag',
 'AU',
 'Angstrom',
 'B',
 'Ba',
 'Barye',
 'Bi',
 'Biot',
 'Bol',
 'Bq',
 'C',
 'Celsius',
 'Ci',
 'CompositeUnit',
 'D',
 'Da',
 'Dalton',
 'Debye',
 'Decibel',
 'DecibelUnit',
 'Dex',
 'DexUnit',
 'EA',
 'EAU',
 'EB',
 'EBa',
 'EC',
 'ED',
 'EF',
 'EG',
 'EGal',
 'EH',
 'EHz',
 'EJ',
 'EJy',
 'EK',
 'EL',
 'EN',
 'EOhm',
 'EP',
 'EPa',
 'ER',
 'ERy',
 'ES',
 'ESt',
 'ET',
 'EV',
 'EW',
 'EWb',
 'Ea',
 'Eadu',
 'Earcmin',
 'Earcsec',
 'Eau',
 'Eb',
 'Ebarn',
 'Ebeam',
 'Ebin',
 'Ebit',
 'Ebyte',
 'Ecd',
 'Echan',
 'Ecount',
 'Ect',
 'Ed',
 'Edeg',
 'Edyn',
 'EeV',
 'Eerg',
 'Eg',
 'Eh',
 'EiB',
 'Eib',
 'Eibit',
 'Eibyte',
 'Ek',
 'El',
 'Elm',
 'Elx',
 'Elyr',
 'Em',
 'Emag',
 'Emin',
 'Emol',
 'Eohm',
 'Epc',
 'Eph',
 'Ephoton',
 'Epix',
 'Epixel',
 'Erad',
 'Es',
 'Esr',
 'Eu',
 'Evox',
 'Evoxel',
 'Eyr',
 'F',
 'Farad',
 'Fr',
 'Franklin',
 'FunctionQuantity',
 'FunctionUnitBase',
 'G',
 'GA',
 'GAU',
 'GB',
 'GBa',
 'GC',
 'GD',
 'GF',
 '

To create a quantity, we multiply a value by a unit.

In [34]:
30 * u.deg

<Quantity 30. deg>

Notice that Jupyter knows how to display units like degrees.

## Selecting a rectangle

Now we'll select a rectangle from -55 to -45 degrees right ascension and -8 to 4 degrees of declination.

In [35]:
phi1_min = -55
phi1_max = -45
phi2_min = -8
phi2_max = 4

To represent a rectangle, I'll create two lists of coordinates and multiply by their units.

In [36]:
phi1_rect = [phi1_min, phi1_min, phi1_max, phi1_max] * u.deg
phi2_rect = [phi2_min, phi2_max, phi2_max, phi2_min] * u.deg

`phi1_rect` and `phi2_rect` represent the coordinates of the corners of a rectangle.  But they are in "[a Heliocentric spherical coordinate system defined by the orbit of the GD1 stream](https://gala-astro.readthedocs.io/en/latest/_modules/gala/coordinates/gd1.html)"

In order to use them in a Gaia query, we have to convert them to ICRS.  We can do that by storing the coordinates in a `GD1Koposov10` object provided by [Gala](https://gala-astro.readthedocs.io/en/latest/coordinates/).

In [37]:
import gala.coordinates as gc

corners = gc.GD1Koposov10(phi1=phi1_rect, phi2=phi2_rect)
type(corners)

gala.coordinates.gd1.GD1Koposov10

In [38]:
corners

<GD1Koposov10 Coordinate: (phi1, phi2) in deg
    [(-55., -8.), (-55.,  4.), (-45.,  4.), (-45., -8.)]>

Now we can use `transform_to` to convert to [International Celestial Reference System](https://en.wikipedia.org/wiki/International_Celestial_Reference_System) coordinates.

In [39]:
import astropy.coordinates as coord

corners_icrs = corners.transform_to(coord.ICRS)
type(corners_icrs)

astropy.coordinates.builtin_frames.icrs.ICRS

In [40]:
corners_icrs

<ICRS Coordinate: (ra, dec) in deg
    [(146.27533314, 19.26190982), (135.42163944, 25.87738723),
     (141.60264825, 34.3048303 ), (152.81671045, 27.13611254)]>

Notice that a rectangle in one coordinate system is not necessarily a rectangle in another.  In this example, the result is a polygon.

## Selecting a polygon

In order to use this polygon as part of an ADQL query, we have to convert it to a string with a comma-separated list of coordinates, as in this example:

```
"""
POLYGON(143.65, 20.98, 
        134.46, 26.39, 
        140.58, 34.85, 
        150.16, 29.01)
"""
```

`corners_icrs` behaves like a list, so we can use a `for` loop to iterate through the points.

In [41]:
for point in corners_icrs:
    print(point)

<ICRS Coordinate: (ra, dec) in deg
    (146.27533314, 19.26190982)>
<ICRS Coordinate: (ra, dec) in deg
    (135.42163944, 25.87738723)>
<ICRS Coordinate: (ra, dec) in deg
    (141.60264825, 34.3048303)>
<ICRS Coordinate: (ra, dec) in deg
    (152.81671045, 27.13611254)>


From that, we can select the coordinates `ra` and `dec`:

In [42]:
for point in corners_icrs:
    print(point.ra, point.dec)

146d16m31.1993s 19d15m42.8754s
135d25m17.902s 25d52m38.594s
141d36m09.5337s 34d18m17.3891s
152d49m00.1576s 27d08m10.0051s


The results are quantities with units, but if we select the `value` part, we get a dimensionless floating-point number.

In [43]:
for point in corners_icrs:
    print(point.ra.value, point.dec.value)

146.27533313607782 19.261909820533692
135.42163944306296 25.87738722767213
141.60264825107333 34.304830296257144
152.81671044675923 27.136112541397996


We can use string `format` to convert these numbers to strings.

In [44]:
point_base = "{point.ra.value}, {point.dec.value}"

t = [point_base.format(point=point)
     for point in corners_icrs]
t

['146.27533313607782, 19.261909820533692',
 '135.42163944306296, 25.87738722767213',
 '141.60264825107333, 34.304830296257144',
 '152.81671044675923, 27.136112541397996']

The result is a list of strings.  We can use `join` to make a single string.

In [45]:
point_list = ', '.join(t)
point_list

'146.27533313607782, 19.261909820533692, 135.42163944306296, 25.87738722767213, 141.60264825107333, 34.304830296257144, 152.81671044675923, 27.136112541397996'

Finally, we can assemble the query.  Here's the base.

In [46]:
query4_base = """SELECT {columns}
FROM gaiadr2.gaia_source
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT(ra, dec), 
                   POLYGON({point_list}))
"""

And here's the result:

In [47]:
query4 = query4_base.format(columns=columns, 
                            point_list=point_list)
print(query4)

SELECT source_id, ra, dec, pmra, pmdec, parallax, parallax_error, radial_velocity
FROM gaiadr2.gaia_source
WHERE parallax < 1
  AND bp_rp BETWEEN -0.75 AND 2 
  AND 1 = CONTAINS(POINT(ra, dec), 
                   POLYGON(146.27533313607782, 19.261909820533692, 135.42163944306296, 25.87738722767213, 141.60264825107333, 34.304830296257144, 152.81671044675923, 27.136112541397996))



As always, we should take a minute to proof-read the query before we launch it.

The results will be bigger than our previous queries, so it will take a little longer.

In [48]:
job4 = Gaia.launch_job_async(query4)
print(job4)

INFO: Query finished. [astroquery.utils.tap.core]
<Table length=140340>
      name       dtype    unit                              description                             n_bad 
--------------- ------- -------- ------------------------------------------------------------------ ------
      source_id   int64          Unique source identifier (unique within a particular Data Release)      0
             ra float64      deg                                                    Right ascension      0
            dec float64      deg                                                        Declination      0
           pmra float64 mas / yr                         Proper motion in right ascension direction      0
          pmdec float64 mas / yr                             Proper motion in declination direction      0
       parallax float64      mas                                                           Parallax      0
 parallax_error float64      mas                                        

Here are the results.

In [49]:
results4 = job4.get_results()
len(results4)

140340

There are more than 100,000 stars in this polygon, but that's a manageable size to work with.

## Saving results

This is the set of stars we'll work with in the next step.  But since we have a substantial dataset now, this is a good time to save it.

Storing the data in a file means we can shut down this notebook and pick up where we left off without running the previous query again.

Astropy `Table` objects provide `write`, which writes the table to disk.

In [50]:
filename = 'gd1_results4.fits'
results4.write(filename, overwrite=True)

Because the filename ends with `fits`, the table is written in the [FITS format](https://en.wikipedia.org/wiki/FITS), which preserves the metadata associated with the table.

If the file already exists, the `overwrite` argument causes it to be overwritten.

To see how big the file is, we can use `ls` with the `-l` option, which prints information about the file including its size in bytes.

In [8]:
!ls -l gd1_results4.fits

-rw-rw-r-- 1 downey downey 8994240 Aug 18 09:36 gd1_results4.fits


The file is about 9 MB.

## Summary

## Best practices

* If you can't download an entire dataset (or it's not practical) use queries to select the data you need.

* Read the metadata and the documentation to make sure you understand the tables, their columns, and what they mean.

* Develop queries incrementally: start with something simple, test it, and add a little bit at a time.

* Use ADQL features like `TOP` and `COUNT` to test before you run a query that might return a lot of data.

* If you know your query will return fewer than 3000 rows, you can run it synchronously, which might complete faster (but it doesn't seem to make much difference).  If it might return more than 3000 rows, you should run it asynchronously.

* ADQL and SQL are not case-sensitive, so you don't have to capitalize the keywords, but you should.

* ADQL and SQL don't require you to break a query into multiple lines, but you should.

* Once you have a query working, save the data in a local file.  If you shut down the notebook and come back to it later, you can reload the file; you don't have to run the query again.

Jupyter notebooks can be good for developing and testing code, but they have some drawbacks.  In particular, if you run the cells out of order, you might find that variables don't have the values you expect.

There are a few things you can do to mitigate these problems:

* Make each section of the notebook self-contained.  Try not to use the same variable name in more than one section.

* Keep notebooks short.  Look for places where you can break your analysis into phases with one notebook per phase.